# Model_v3 Max length variation

## Generate t2t data

#### When generate the t2t dataset by running "generate_problem.py", 
#### reviews that the summary contains exactly same sentences with review text are dropped out.

In [14]:
# self-defined 'generate_problem.py' file in the same folder.
# define the directory for generated data
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
import generate_problem_new

USR_DIR= './'
PROBLEM = 'OnlineRevewProjectUSYD_new'
TMP_DIR = './tmp' 
DATA_LOC = './data' 

# Generated training data
t2t_problem = generate_problem_new.OnlineRevewProjectUSYD_new(PROBLEM)
t2t_problem.generate_data(DATA_LOC, TMP_DIR) 

INFO:tensorflow:Found vocab file: ./data/vocab.online_revew_project_usyd_new.32768.subwords
INFO:tensorflow:Skipping generator because outputs files exists at ['./data/online_revew_project_usyd_new-unshuffled-train-00000-of-00100', './data/online_revew_project_usyd_new-unshuffled-train-00001-of-00100', './data/online_revew_project_usyd_new-unshuffled-train-00002-of-00100', './data/online_revew_project_usyd_new-unshuffled-train-00003-of-00100', './data/online_revew_project_usyd_new-unshuffled-train-00004-of-00100', './data/online_revew_project_usyd_new-unshuffled-train-00005-of-00100', './data/online_revew_project_usyd_new-unshuffled-train-00006-of-00100', './data/online_revew_project_usyd_new-unshuffled-train-00007-of-00100', './data/online_revew_project_usyd_new-unshuffled-train-00008-of-00100', './data/online_revew_project_usyd_new-unshuffled-train-00009-of-00100', './data/online_revew_project_usyd_new-unshuffled-train-00010-of-00100', './data/online_revew_project_usyd_new-unshuffled

In [ ]:
# Generated 3637767 Examples

## Build and Train model

## Max len full

In [18]:
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_v2'

### Define Hyper params

In [11]:
# Define hparams
hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 
hparams.learning_rate = 0.1 
hparams.num_hidden_layers = 6
# hparams.num_encoder_layers = 2 ###
# hparams.num_decoder_layers = 2 ###
hparams.dropout =0.4
hparams.num_heads = 8 #16-Large- # default8
hparams.attention_dropout = 0.5 ##
hparams.max_input_seq_length = 1024
# hparams.shared_embedding_and_softmax_weights = False

decode_hp = decoding.decode_hparams()
decode_hp.return_beams = True # False
decode_hp.beam_size = 2

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))

{'hidden_size': 512, 'max_length': 0, 'min_length_bucket': 8, 'relu_dropout_broadcast_dims': '', 'vocab_divisor': 1, 'scheduled_sampling_prob': 0.0, 'nbr_decoder_problems': 1, 'attention_key_channels': 0, 'pos': 'timing', 'attention_value_channels': 0, 'length_bucket_step': 1.1, 'moe_num_experts': 16, 'relu_dropout': 0.1, 'learning_rate_decay_scheme': 'noam', 'dropout': 0.4, 'optimizer': 'Adam', 'multiproblem_schedule_threshold': 0.5, 'use_pad_remover': True, 'learning_rate': 0.1, 'optimizer_adafactor_beta2': 0.999, 'multiproblem_max_target_length': -1, 'ffn_layer': 'dense_relu_dense', 'filter_size': 2048, 'symbol_modality_num_shards': 16, 'label_smoothing': 0.1, 'min_length': 0, 'multiproblem_fixed_train_length': -1, 'kernel_width': 1, 'symbol_modality_skip_top': False, 'prepend_mode': 'prepend_inputs_masked_attention', 'multiply_embedding_mode': 'sqrt_depth', 'use_target_space_embedding': True, 'attention_dropout': 0.5, 'optimizer_momentum_momentum': 0.9, 'optimizer_zero_grads': Fals

### Build and train model

In [13]:
# create run config
RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=1000,
    )

t2t_model = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        model_name=MODEL,
        problem_name=t2t_problem, 
        data_dir=DATA_LOC, 
        train_steps=10001,
        eval_steps=100,
        min_eval_frequency=1000,
        eval_use_test_set=True
    )

hparams.problem_hparams.was_reversed =False ###
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model.train_and_evaluate()


INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']


/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


ValueError: Hyperparameter name is reserved: data_dir

## (skip) Another example - Max len 1024 

In [17]:
# select the transformer model

from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding, metrics
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json
import timeit
# running time calculation
start = timeit.default_timer()

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_1024_new'

# Define hparams
hparams = create_hparams(HPARAMS)
hparams.batch_size = 1024 #512 
hparams.learning_rate = 0.1
hparams.num_hidden_layers = 6
# hparams.num_encoder_layers = 2
# hparams.num_decoder_layers = 2 
hparams.dropout =0.2
hparams.max_input_seq_length = 1024 #####
hparams.shared_embedding_and_softmax_weights = False ###????

decode_hp = decoding.decode_hparams()
decode_hp.return_beams = True # False
decode_hp.beam_size = 2

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))

# create run config
RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=1000,
    )
# t2t_problem.eval_metrics()

t2t_model_1024_new = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        model_name=MODEL,
        problem_name=t2t_problem, 
        data_dir='./data', 
        train_steps=40001,
        eval_steps=100,
        min_eval_frequency=1000,
        eval_use_test_set=True
    )

hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_1024_new.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

{'eval_run_autoregressive': False, 'multiproblem_schedule_threshold': 0.5, 'proximity_bias': False, 'max_input_seq_length': 1024, 'moe_k': 2, 'attention_dropout': 0.1, 'multiproblem_reweight_label_loss': False, 'optimizer_momentum_nesterov': False, 'initializer_gain': 1.0, 'eval_drop_long_sequences': False, 'attention_dropout_broadcast_dims': '', 'initializer': 'uniform_unit_scaling', 'multiproblem_label_weight': 0.5, 'optimizer_adafactor_decay_type': 'pow', 'multiproblem_schedule_max_examples': 10000000.0, 'video_num_target_frames': 1, 'optimizer_adafactor_beta1': 0.0, 'layer_prepostprocess_dropout': 0.1, 'multiproblem_max_target_length': -1, 'symbol_modality_num_shards': 16, 'clip_grad_norm': 0.0, 'learning_rate_constant': 1.0, 'attention_variables_3d': False, 'optimizer_adafactor_multiply_by_parameter_scale': True, 'weight_dtype': 'float32', 'length_bucket_step': 1.1, 'daisy_chain_variables': True, 'batch_shuffle_size': 512, 'multiproblem_max_input_length': -1, 'optimizer_adafactor_

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INFO:tensorflow:partition: 0 num_data_files: 100
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Base learning rate: 0.100000
INFO:tensorflow:Trainable Variables Total size: 105644544
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


KeyboardInterrupt: 

#  ------Decoding Check------

In [19]:
import tensorflow as tf
ckpt_path = tf.train.latest_checkpoint("./model_files_v2") #model.ckpt
ckpt_path

'./model_files_v2/model.ckpt-10001'

In [20]:
import tensorflow as tf

MODEL = 'transformer'
HPARAMS = 'transformer_prepend'
TRAIN_DIR = 'model_files_v2'

hparams = create_hparams(HPARAMS, data_dir=DATA_LOC, problem_name=t2t_problem)
print(hparams.problem_hparams)

# decoder hyperparam
decode_hp = decoding.decode_hparams()
decode_hp.return_beams = False #True # False
# decode_hp.beam_size = 2


RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

estimator = trainer_lib.create_estimator(
    MODEL,
    hparams,
    RUN_CONFIG, #t2t_trainer.create_run_config(hparams),
    decode_hparams=decode_hp)

# main
decoding.decode_interactively(estimator, hparams, decode_hp, checkpoint_path=ckpt_path)

[('batch_size_multiplier', 1), ('input_space_id', 0), ('loss_multiplier', 1.0), ('modality', {'targets': <tensor2tensor.layers.modalities.SymbolModality object at 0x7fd7f956ce10>, 'inputs': <tensor2tensor.layers.modalities.SymbolModality object at 0x7fd7f956ce48>}), ('stop_at_eos', 1), ('target_space_id', 0), ('vocabulary', {'targets': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7fd898aaa5c0>, 'inputs': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder object at 0x7fd898aaa5c0>}), ('was_copy', False), ('was_reversed', True)]
INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_train_distribute': None, '_protocol': None, '_task_type': None, '_evaluation_master': '', 'use_t

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INTERACTIVE MODE  num_samples=1  decode_length=100  
  it=<input_type>     ('text' or 'image' or 'label', default: text)
  ns=<num_samples>    (changes number of samples, default: 1)
  dl=<decode_length>  (changes decode length, default: 100)
  <source_string>                (decode)
  q                   (quit)
>i origonally did not get the item i ordered.  when contacting the company they got back with me quickly.  it turned out the item was out of stock.  they sent it once it came in stock.  this was not a problem for me since i ordered the item way in advance for my daughters birthday.  i would recommend this selling but ordered ahead of time just incase.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_functio

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model_files_v2/model.ckpt-10001


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key transformer/symbol_modality_40055_512/shared/weights_0 not found in checkpoint
	 [[node save/RestoreV2_1 (defined at /usr/local/lib/python3.5/dist-packages/tensorflow_estimator/python/estimator/estimator.py:627) ]]
	 [[node save/RestoreV2_1 (defined at /usr/local/lib/python3.5/dist-packages/tensorflow_estimator/python/estimator/estimator.py:627) ]]

Caused by op 'save/RestoreV2_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packa

In [21]:
# sample:

review = '''
i origonally did not get the item i ordered.  when contacting the company they got back with me quickly.  it turned out the item was out of stock.  they sent it once it came in stock.  this was not a problem for me since i ordered the item way in advance for my daughters birthday.  i would recommend this selling but ordered ahead of time just incase.
'''
summary = '''
happy with purchase even though it came a lot later than expected.
'''

### Decoder example -wrong

In [69]:

review1 = '''
we have many of the old , old issue but the number had depleted there were not enough books to allow us to use them regularly with the additional supply the books will be used more often they are a good old standby for gospel singing
'''
review2 = '''
i origonally did not get the item i ordered.  when contacting the company they got back with me quickly.  it turned out the item was out of stock.  they sent it once it came in stock.  this was not a problem for me since i ordered the item way in advance for my daughters birthday.  i would recommend this selling but ordered ahead of time just incase.
'''
s2 = '''
happy with purchase even though it came a lot later than expected.
'''

summary = summarize(review2)

print("REVIEWS: %s" % review2)
print("PRED_SUMMARY: %s" % summary)
print("GOLD_SUMMARY: %s" % s2)


REVIEWS: 
i origonally did not get the item i ordered.  when contacting the company they got back with me quickly.  it turned out the item was out of stock.  they sent it once it came in stock.  this was not a problem for me since i ordered the item way in advance for my daughters birthday.  i would recommend this selling but ordered ahead of time just incase.

PRED_SUMMARY: read 10; i the the did not get the item i ordered.  when its company they got back with me me quickly.  it turned out the item was out of stock.  they sent it once it came in stock.  this was not a problem for me since i ordered the item way in advance for my daughters birthday.  i would recommend this selling but ordered ahead of time just incase the has 10;
GOLD_SUMMARY: 
happy with purchase even though it came a lot later than expected.

